# Космический корабль Титаник

[Spaceship Titanic: predict which passengers are transported to an alternate dimension](https://www.kaggle.com/competitions/spaceship-titanic/)  

Задача взята из соревнования на kaggle для начинающих. Представляет собой классическую бинарную классификацию по мотивам известного датасета Titanic.  

Суть датасета и задачи заключается в следующем: космический корабль Титаник, двигавшийся к далёким звездным системам, попал в пространственно-временную аномалию, вследствие чего половина пассажиров попали в альтернативную реальность. По данным бортового компьютера **необходимо предсказать, попал ли тот или иной пассажир в эту самую параллельную реальность**, тем самым облегчив спасательную операцию.  



Данные представлены двумя файлами. В файле `train.csv` представлена информация о 8693 пассажирах в 14 колонках. В файле `test.csv` представлена информация о 4277 пассажирах с теми же колонками, но без целевой переменной. Собственно эти тестовые данные и нужно классифицировать, отправив на странице соревнования файл `submission.csv` с двумя колонками: `PassengerId` и `Transported`.  

**Описание** датасета:  

* `PassengerId` - Уникальный идентификатор пассажира, состоящий из двух частей `gggg_pp`, где `gggg` - это номер группы пассажира, а `pp` - номер самого пассажира в группе, люди в одной группе могут быть связаны, например, родством.  
* `HomePlanet` - Планета, с которой пассажир отбыл.  
* `CryoSleep` - Был ли пассажир в состоянии анабиоза.  
* `Cabin` - Номер каюты пассажира, состоит из трёх частей `deck/num/side` - палуба/номер/сторона, где сторона может быть или `P` для левого борта или `S` для правого.  
* `Destination` - Планета назначения, куда летит пассажир.  
* `Age` - Возраст.  
* `VIP` - Заплатил ли пассажир за VIP услуги.  
* `RoomService`, `FoodCourt`, `ShoppingMall`, `Spa`, `VRDeck` - Сколько заплатил пассажир за те или иные роскошные удобства.  
* `Name` - Имя и фамилия.  
* `Transported` - Был ли пассажир отправлен в альтернативное измерение. Это **целевая переменная**.  

In [1]:
# импорт необходимых библиотек 

from os.path import join as path_join
import pandas as pd
import numpy as np
import numpy.typing as npt
from typing import Union

import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from catboost import CatBoostClassifier

Matplotlib is building the font cache; this may take a moment.


: 

: 

In [ ]:
# Установим некоторые константы 

TEST_SIZE = 0.2
RANDOM_STATE = 42

In [2]:
# Загрузим данные

train_dataset = pd.read_csv(path_join('data', 'train.csv'))
test_dataset = pd.read_csv(path_join('data', 'test.csv'))

print('Размер тренировочного датасета:', train_dataset.shape)
print('Размер тестового датасета:', test_dataset.shape)
train_dataset.head()

Размер тренировочного датасета: (8693, 14)
Размер тестового датасета: (4277, 13)


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


## EDA - разведочный анализ данных  

### Основная информация о датасете и признаках  

In [3]:
df = train_dataset.copy()

(8693, 14)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [5]:
df.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [6]:
df.describe(include=['object', 'bool'])

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,VIP,Name,Transported
count,8693,8492,8476,8494,8511,8490,8493,8693
unique,8693,3,2,6560,3,2,8473,2
top,0001_01,Earth,False,G/734/S,TRAPPIST-1e,False,Gollux Reedall,True
freq,1,4602,5439,8,5915,8291,2,4378


### Гипотезы  

* Имя и фамилия вряд ли повлияют на окончательный результат.  
* В численных колонках есть выбросы.  
* Категориальные переменные распределены по разному и где-нибудь есть дисбаланс, который стоит учитывать при разбиении датасета на тренировочную и тестовую выборки.  
* Коррелируемых признаков нет.  

In [7]:
# посмотрим, стоит ли как-то использовать имена

df['FirstName'] = df['Name'].apply(lambda x: None if pd.isna(x) else x.split()[0])
df['LastName'] = df['Name'].apply(lambda x: None if pd.isna(x) else x.split()[1])

df[['FirstName', 'LastName']].describe()

# уникальных значений имен и фамилий слишком много 
# и маловероятно, что они будут полезны, 
# так что обойдёмся без них 

,FirstName,LastName
count,8493,8493
unique,2706,2217
top,Idace,Casonston
freq,13,18


In [8]:
# посмотрим на данные в колонке Cabin

df['CabinDeck'] = df['Cabin'].apply(lambda x: None if pd.isna(x) else x.split('/')[0])
df['CabinNumber'] = df['Cabin'].apply(lambda x: None if pd.isna(x) else x.split('/')[1])
df['CabinSide'] = df['Cabin'].apply(lambda x: None if pd.isna(x) else x.split('/')[2])

max_num = df['CabinNumber'].dropna().astype(int).max()
print('Максимальное значение в CabinNumber:', max_num)
df[['CabinDeck', 'CabinNumber', 'CabinSide']].describe()

# пропуски в CabinDeck и CabinSide можно заполнить самым частым значением 
# а пропуски в CabinNumber неудобно заполнить ни средним, ни самым частым значением 
# поэтому заполним его новым значением - тем, что больше макисмального на 1

Максимальное значение в CabinNumber: 1894


,CabinDeck,CabinNumber,CabinSide
count,8494,8494,8494
unique,8,1817,2
top,F,82,S
freq,2794,28,4288


In [9]:
# посмотрим корреляцию признаков

corr = df.corr()

corr

# коррелируемых признаков нет

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
Age,1.000000,0.068723,0.130421,0.033133,0.123970,0.101007,-0.075026
RoomService,0.068723,1.000000,-0.015889,0.054480,0.010080,-0.019581,-0.244611
FoodCourt,0.130421,-0.015889,1.000000,-0.014228,0.221891,0.227995,0.046566
ShoppingMall,0.033133,0.054480,-0.014228,1.000000,0.013879,-0.007322,0.010141
Spa,0.123970,0.010080,0.221891,0.013879,1.000000,0.153821,-0.221131
VRDeck,0.101007,-0.019581,0.227995,-0.007322,0.153821,1.000000,-0.207075
Transported,-0.075026,-0.244611,0.046566,0.010141,-0.221131,-0.207075,1.000000


## Data Preparation  

Преобразование данных будет проводиться через pipeline библиотеки sklearn.  

In [4]:
class DataFrameColumnTransformer(ColumnTransformer): 
    """Кастомный трансформер колонок, который возвращает pd.DataFrame, а не массив numpy."""

    def get_names(self) -> list[str]:
        names = self.get_feature_names_out()
        names = [n.split('__')[-1] for n in names]
        return names

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        result = super().transform(X)
        names = self.get_names()
        for i, name in enumerate(names):
            X[name] = result[:, i]
            X[name] = X[name].astype(float, errors='ignore')
        return X
    
    def fit_transform(self, X: pd.DataFrame, y: pd.Series | None = None) -> pd.DataFrame:
        result = super().fit_transform(X, y)
        names = self.get_names()
        for i, name in enumerate(names):
            X[name] = result[:, i]
            X[name] = X[name].astype(float, errors='ignore')
        return X

### Feature Engineering - создание признаков  

Информация в некоторых колонках содержит больше одного признака. Их стоит разделить.  
Колонку `PassengerId` можно разделить на группу `PassengerGroup` и номер пассажира `PassengerNumber`.  
Колонку `Cabin` можно разделить на палубу `CabinDeck`, номер `CabinNumber` и сторону `CabinSide`.  

Можно создать признак `AgeCategory`, показывающий возрастную группу пассажира.  
Также было бы неплохо создать признак, содержащий общую сумму трат `TotalExpenses`.  

Созданные признаки привести к нужным типам.  

In [5]:
def create_features(data: pd.DataFrame) -> pd.DataFrame:
    """Создаёт новые признаки: 
    * `PassengerGroup` и `PassengerNumber` из `PassengerId`; 
    * `CabinDeck`, `CabinNumber` и `CabinSide` из `Cabin`; 
    * `TotalExpenses` как сумма `RoomService`, `FoodCourt`, `ShoppingMall`, `Spa` и `VRDeck`.
    """

    data = data.copy()

    # Splitting columns PassengerId and Cabin
    data[['PassengerGroup', 'PassengerNumber']] = data['PassengerId'].str.split('_', expand=True)
    data[['CabinDeck', 'CabinNumber', 'CabinSide']] = data['Cabin'].str.split('/', expand=True)

    # Creating AgeCategory feature 
    def get_age_category(age: float) -> str: 
        if age < 0: 
            return float('nan')
        elif age <= 25: 
            return 'young'
        elif age <= 50: 
            return 'average'
        else: 
            return 'old'
    data['AgeCategory'] = data['Age'].apply(get_age_category)

    # Creating TotalExpenses feature
    columns_for_summing = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    data['TotalExpenses'] = data[columns_for_summing].fillna(0.0).sum(axis=1)    

    # Set new features as float type
    columns_to_float = ['PassengerGroup', 'PassengerNumber', 'CabinNumber']
    data[columns_to_float] = data[columns_to_float].astype(float)

    return data

features_creator = FunctionTransformer(create_features)

### Заполнение пропусков  

Во всех колонках, кроме id пассажира и целевой переменной, есть пропуски.  
Пропуски в колонке `Age` можно заполнить средним значением.  
Пропуски в колонках `HomePlanet`, `CryoSleep`, `Destination`, `VIP`, `CabinDeck`, `CabinSide`, `AgeCategory` можно заполнить самым частым значением.  
Пропуски в колонках `RoomService`, `FoodCourt`, `ShoppingMall`, `Spa`, `VRDeck` можно заполнить 0, как самым частым значением, потому что роскошные удобства может себе позволить лишь небольшой круг людей.  
Пропуски в колонке `CabinNumber` можно заполнить новым значением, большим максимального на 1.  

In [6]:
class MaxMoreImputer(BaseEstimator, TransformerMixin):
    """Класс для заполнения пропущенных значений. 
    Пустые значения заполняются максимальным значением колонки, увеличенных на значение `step`.
    """

    def __init__(self, step: float = 1.0) -> None:
        super().__init__()
        self.step = step
        self._features_names = None
        self._impute_value = None
    
    def fit(self, X: npt.ArrayLike, y: npt.ArrayLike | None = None) -> 'self':
        self._impute_value = X.astype(float).max()
        self._features_names = X.columns
        return self
    
    def transform(self, X: npt.ArrayLike, y: npt.ArrayLike | None = None) -> np.ndarray:
        check_is_fitted(self, '_impute_value')
        return X.fillna(self._impute_value + self.step)
    
    def get_feature_names_out(self, input_features: npt.ArrayLike | None = None) -> np.ndarray:
        check_is_fitted(self, '_features_names')
        return self._features_names

In [ ]:
missings_filler = DataFrameColumnTransformer(transformers=[
    ('age_imputer', SimpleImputer(strategy='mean'), ['Age']), 
    ('mode_imputer', SimpleImputer(strategy='most_frequent'), 
     ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'CabinDeck', 'CabinSide', 'AgeCategory']), 
    ('zero_imputer', SimpleImputer(strategy='constant', fill_value=0.0), 
     ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']), 
    ('cabin_number_imputer', MaxMoreImputer(), ['CabinNumber']), 
], remainder='passthrough')

### Обработка выбросов  

Обработка выбросов в колонке `Age`: те значения, что меньше нижней границы интерквартального размаха, приравнять к этой нижней границе, те, что выше - к верхней границе.  
Искать выбросы в колонках `RoomService`, `FoodCourt`, `ShoppingMall`, `Spa`, `VRDeck` без учёта нулевого значения.  

Может быть произвести логарифмирование?

In [ ]:
# Пока ничего, нужно провести анализ сперва

### Нормализация численных переменных  

Стандартизируем числовые признаки с помощью StandardScaler.  

In [ ]:
# используется column transformer, чтобы разделить 
# шаги нормализации и кодирования категориальных признаков

normalizer = DataFrameColumnTransformer(transformers=[
    ('scaler', StandardScaler(), 
     ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 
      'PassengerGroup', 'PassengerNumber', 'CabinNumber', 'TotalExpenses']),  
], remainder='passthrough')

### Кодирование категориальных переменных  



In [ ]:
# используется column transformer, чтобы разделить 
# шаги нормализации и кодирования категориальных признаков

onehot_encoder = DataFrameColumnTransformer(transformers=[
    ('encoder', OneHotEncoder(sparse=False, handle_unknown='ignore', drop='if_binary'), 
     ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'CabinDeck', 'CabinSide', 'AgeCategory']), 
], remainder='passthrough')

### Удаление ненужных колонок  



In [ ]:
def remove_columns(data: pd.DataFrame) -> pd.DataFrame:
    """Удаляет ненужные колонки: 
    `PassengerId`, `HomePlanet`, `CryoSleep`, `Cabin`, `Destination`, `VIP`, `Name`, `CabinDeck`, `CabinSide`, `AgeCategory`.
    """

    data = data.copy()

    columns_to_drop = ['PassengerId', 'Cabin', 'Name', 'HomePlanet', 'Destination', 
                       'CabinDeck', 'CabinSide', 'AgeCategory', 'CryoSleep', 'VIP']
    data = data.drop(columns=columns_to_drop)

    return data

columns_remover = FunctionTransformer(remove_columns)

### Соберём пайплайн  

Соберём все запланированные шаги подготовки данных в единый пайплайн.  

In [ ]:
x = train_dataset.copy() 
y = x.pop('Transported')

In [ ]:
preprocessor = Pipeline(steps=[
    ('feature_engineering', features_creator), 
    ('filling_missing', missings_filler), 
    # ('outliers_handling', ), 
    ('normalization', normalizer), 
    ('onehot_encoding', onehot_encoder), 
    ('feature_selection', columns_remover), 
])

## Modeling  

### Разделение данных  

In [ ]:
x = train_dataset.copy()
y = x.pop('Transported')

x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)
print(f'train shapes: {x_train.shape} and {y_train.shape}')
print(f'valid shapes: {x_valid.shape} and {y_valid.shape}')

train shapes: (6954, 13) and (6954,)
valid shapes: (1739, 13) and (1739,)


### Baseline  

В качестве бейзлайн модели возъмём простую модель k-ближайших соседей.  

In [ ]:
baseline_model = Pipeline(steps=[
    ('preprocessor', preprocessor), 
    ('model', KNeighborsClassifier()), 
])

baseline_scores = cross_val_score(baseline_model, x_train, y_train)
print(f'baseline_scores: mean={baseline_scores.mean()}, std={baseline_scores.std()}')

baseline_scores: mean=0.7752377307356129, std=0.003450765605891145


### Обучение нескольких моделей  

Обучим несколько моделей с параметрами по умолчанию, выберем лучшую из них и уже её будем оптимизировать далее.  

models = [
    # CatBoostClassifier
    # RandomForest 
    # LogisticRegression? 
    # 
]

In [ ]:
catboost_model = Pipeline(steps=[
    ('preprocessor', preprocessor), 
    ('model', CatBoostClassifier()), 
])

catboost_scores = cross_val_score(catboost_model, x_train, y_train)
print(f'catboost_scores: mean={catboost_scores.mean()}, std={catboost_scores.std()}')